### Import necessary packages

In [1]:
import os
import pickle
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
# from sklearn.utils.class_weight import compute_class_weight
# from imblearn.over_sampling import SMOTE

from tensorflow.keras.models import Sequential
from tensorflow.keras import Input
from tensorflow.keras.layers import GRU, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from imblearn.over_sampling import SMOTE
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

from Toolshed import Predictions

# Only use tensorflow in CPU mode
import tensorflow as tf
tf.config.set_visible_devices([],'GPU')
%load_ext tensorboard
import tensorboard



2024-08-14 10:47:26.615106: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-14 10:47:37.921108: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


### Define site variables and set up directory

In [ ]:
# Name of site to save directory and files under
sitename = 'StAndrewsEastS2Full'
filepath = os.path.join(os.getcwd(), 'Data')

# Load in transect data with coastal change variables
TransectInterGDF, TransectInterGDFWater, TransectInterGDFTopo, TransectInterGDFWave = Predictions.LoadIntersections(filepath, sitename)
# Compile relevant coastal change metrics into one dataframe
CoastalDF = Predictions.CompileTransectData(TransectInterGDF, TransectInterGDFWater, TransectInterGDFTopo, TransectInterGDFWave)



### Prepare data for training
Ensure each of the variables are synchronised by timestep

In [14]:
TransectIDs = [50]

for Tr in TransectIDs:
    TransectDF = CoastalDF.iloc[[Tr],:] # single-row dataframe
print(TransectDF)

    TransectID                                              dates  \
50          50  [2015-08-07, 2015-08-17, 2015-11-22, 2016-02-1...   

                                            distances  \
50  [137.069698951171, 144.04709852809864, 138.324...   

                                           wlcorrdist  \
50  [509.7608836456035, 181.57936499534543, 440.88...   

                                            waterelev  \
50  [-2.079662590257345, 0.5471072274658595, -1.75...   

                                           beachwidth  \
50  [365.7137851175048, 43.25529176978043, 295.990...   

                                              TZwidth  TZwidthMn   SlopeMax  \
50  [10.196233442125981, 10.196233442125981, 10.19...  15.273152  29.881193   

    SlopeMean                                             WaveHs  \
50  10.838753  [0.25638099458485064, 0.16007945575151394, 0.8...   

                                              WaveDir  \
50  [91.71905048709071, 70.2255225279233, 57.960

### Feature engineering
Time-lag the features to capture temporal dependencies and aggregate features over a moving window

### Define model

RNN model may need attention layer to help focus on important parts of timeseries (i.e. storm/high-risk events)
Model architecture:
- Input layers: 
  - Historic obervations layer [shape = (batch_size, timesteps, num of variables)]
  - Forecast data layer [shape = (batch_size, forecast_steps, num of variables)]
- Feature fusion layer:
  - Concatenate two LSTMs into one single feature vector
  - Pass to fully connected dense layer
- Output layer:
  - Softmax layer for generating multiclass of risk levels
  - Define model by compiling (using Adam optimiser and categorical cross-entropy loss)

### Separate data into train/validate/test data
Goal is to minimise differnece between predicted and actually labelled risk levels, which are based on past/current state and strength of forecast severity

### Prediction
Incorporate in forecast data so that newest forecast will guide the overall risk classification